In [ ]:
import tushare as ts

In [ ]:
ts.set_token('3463f0f3c5113c101cd9831597c579b4e429e5751f3f1739c6af830a')

In [ ]:
pro = ts.pro_api()

600519.SH

贵州茅台


In [ ]:
df1 = pro.daily(ts_code='600519.SH', end_date='20250722')

In [ ]:
df1

In [ ]:
type(df1)

In [ ]:
df1.to_csv('600519.SH.csv', index=False)

In [ ]:
ndsd = pro.daily(ts_code='300750.SZ', end_date='20250722')

In [ ]:
ndsd

In [ ]:
ndsd.to_csv('ndsd_300750.SZ.csv', index=False)

In [ ]:
joyson = pro.daily(ts_code='600699.SH', end_date='20250722')
joyson.to_csv('joyson_600699.SH.csv', index=False)

In [ ]:
tpjt = pro.daily(ts_code='601689.SH', end_date='20250722')
tpjt.to_csv('tpjt_601689.SH.csv', index=False)

In [ ]:
pinganbank = pro.daily(ts_code='000001.SZ', end_date='20250722')
pinganbank.to_csv('p_000001.SZ.csv', index=False)

In [2]:
import akshare as ak

print(ak.__version__)

1.17.31


In [4]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20250722", end_date='20250724', adjust="")

In [5]:
stock_zh_a_hist_df

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2025-07-22,600699,17.71,17.73,17.79,17.62,235707,416989247.0,0.96,-0.28,-0.05,1.72
1,2025-07-23,600699,17.69,17.92,18.12,17.60,419966,752988053.0,2.93,1.07,0.19,3.07
2,2025-07-24,600699,17.95,17.98,18.22,17.89,335495,604135187.0,1.84,0.33,0.06,2.45


In [ ]:
a1

In [ ]:
joyson = pro.daily(ts_code='600699.SH', start_date="20250722", end_date='20250724',)

In [ ]:
joyson

In [6]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20250722", end_date='20250724', adjust="")

In [7]:
stock_hk = ak.stock_hk_hist_min_em()

In [8]:
stock_hk

,时间,开盘,收盘,最高,最低,成交量,成交额,最新价
0,2025-08-05 09:30:00,0.00,4.26,4.26,4.26,0,0.0,4.2600
1,2025-08-05 09:31:00,0.00,4.29,4.29,4.24,3500,14915.0,4.2614
2,2025-08-05 09:32:00,0.00,4.29,4.29,4.29,0,0.0,4.2614
3,2025-08-05 09:33:00,0.00,4.29,4.32,4.29,54000,232210.0,4.2978
4,2025-08-05 09:34:00,0.00,4.34,4.35,4.31,15000,64980.0,4.3048
...,...,...,...,...,...,...,...,...
1650,2025-08-11 15:56:00,5.04,5.04,5.04,5.04,3500,17640.0,5.0084
1651,2025-08-11 15:57:00,5.04,5.05,5.05,5.04,29500,148895.0,5.0085
1652,2025-08-11 15:58:00,5.06,5.06,5.06,5.06,12000,60720.0,5.0085
1653,2025-08-11 15:59:00,5.06,5.08,5.08,5.05,54500,276300.0,5.0089


In [ ]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20000000", end_date='20250724', adjust="")

In [ ]:
stock_zh_a_hist_df

In [ ]:
joyson = pro.daily(ts_code='600699.SH', start_date="20000000", end_date='20250724',)

In [ ]:
joyson

In [9]:
import pandas as pd
import torch
import numpy as np

def normalize_stock_data(df, features):
    """
    对股票数据进行Min-Max归一化，按股票代码分组处理
    
    参数:
    df: 包含多只股票数据的DataFrame，需包含'股票代码'列
    features: 需要归一化的特征列表
    
    返回:
    normalized_df: 归一化后的DataFrame
    scalers: 保存每只股票每个特征的min和max，用于后续反归一化
    """
    # 初始化存储归一化参数的字典
    scalers = {}
    
    # 复制原始数据，避免修改原数据
    normalized_df = df.copy()
    
    # 按股票代码分组处理
    for stock_code, group in df.groupby('股票代码'):
        scalers[stock_code] = {}
        
        # 对每个特征进行归一化
        for feature in features:
            # 获取该股票该特征的最小值和最大值
            min_val = group[feature].min()
            max_val = group[feature].max()
            
            # 处理极端情况（最大值等于最小值时避免除零）
            if max_val == min_val:
                normalized = np.zeros_like(group[feature].values)
            else:
                # Min-Max归一化公式: (x - min) / (max - min)
                normalized = (group[feature] - min_val) / (max_val - min_val)
            
            # 保存归一化参数
            scalers[stock_code][feature] = {
                'min': min_val,
                'max': max_val
            }
            
            # 更新归一化后的值
            normalized_df.loc[group.index, feature] = normalized
    
    # 将归一化后的数据转换为PyTorch张量（可选）
    tensor_data = torch.tensor(normalized_df[features].values, dtype=torch.float32)
    
    return normalized_df, scalers, tensor_data

# 示例用法
if __name__ == "__main__":
    # 1. 准备示例数据（实际使用时替换为你的数据）
    data = {
        '日期': ['2025-07-22', '2025-07-23', '2025-07-24', 
                '2025-07-22', '2025-07-23'],
        '开盘价': [17.71, 17.69, 17.95, 5.20, 5.30],
        '收盘价': [17.73, 17.92, 17.98, 5.15, 5.40],
        '最高价': [17.79, 18.12, 18.22, 5.30, 5.50],
        '最低价': [17.62, 17.60, 17.89, 5.10, 5.25],
        '成交量': [235707, 419966, 335495, 100000, 150000],
        '成交额': [416989247.0, 752988053.0, 604135187.0, 500000.0, 800000.0],
        '振幅': [0.96, 2.93, 1.84, 3.85, 4.76],
        '涨跌幅': [-0.28, 1.07, 0.33, -1.00, 4.85],
        '涨跌额': [-0.05, 0.19, 0.06, -0.05, 0.25],
        '股票代码': ['600699', '600699', '600699', '600000', '600000']
    }
    df = pd.DataFrame(data)
    
    # 2. 定义需要归一化的特征
    features_to_normalize = ['开盘价', '收盘价', '最高价', '最低价', 
                            '成交量', '成交额', '振幅', '涨跌幅', '涨跌额']
    
    # 3. 执行归一化
    normalized_df, scalers, tensor_data = normalize_stock_data(df, features_to_normalize)
    
    # 4. 输出结果
    print("归一化前的数据:\n", df[['股票代码'] + features_to_normalize].head())
    print("\n归一化后的数据:\n", normalized_df[['股票代码'] + features_to_normalize].head())
    print("\n归一化参数示例（600699的收盘价）:\n", scalers['600699']['收盘价'])
    print("\n转换为PyTorch张量的形状:", tensor_data.shape)


归一化前的数据:
      股票代码    开盘价    收盘价    最高价    最低价     成交量          成交额    振幅   涨跌幅   涨跌额
0  600699  17.71  17.73  17.79  17.62  235707  416989247.0  0.96 -0.28 -0.05
1  600699  17.69  17.92  18.12  17.60  419966  752988053.0  2.93  1.07  0.19
2  600699  17.95  17.98  18.22  17.89  335495  604135187.0  1.84  0.33  0.06
3  600000   5.20   5.15   5.30   5.10  100000     500000.0  3.85 -1.00 -0.05
4  600000   5.30   5.40   5.50   5.25  150000     800000.0  4.76  4.85  0.25

归一化后的数据:
      股票代码       开盘价   收盘价       最高价       最低价       成交量       成交额        振幅  \
0  600699  0.076923  0.00  0.000000  0.068966  0.000000  0.000000  0.000000   
1  600699  0.000000  0.76  0.767442  0.000000  1.000000  1.000000  1.000000   
2  600699  1.000000  1.00  1.000000  1.000000  0.541564  0.556984  0.446701   
3  600000  0.000000  0.00  0.000000  0.000000  0.000000  0.000000  0.000000   
4  600000  1.000000  1.00  1.000000  1.000000  1.000000  1.000000  1.000000   

        涨跌幅       涨跌额  
0  0.000000  0.000

In [11]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20250522", end_date='20250724', adjust="")

In [12]:
stock_zh_a_hist_df

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2025-05-22,600699,20.00,19.38,20.32,19.35,906208,1.782245e+09,4.83,-3.53,-0.71,6.62
1,2025-05-23,600699,19.58,19.23,20.04,19.11,721013,1.407556e+09,4.80,-0.77,-0.15,5.27
2,2025-05-26,600699,19.31,18.96,19.37,18.87,430418,8.185419e+08,2.60,-1.40,-0.27,3.15
3,2025-05-27,600699,18.90,18.51,19.00,18.23,426355,7.866479e+08,4.06,-2.37,-0.45,3.12
4,2025-05-28,600699,18.21,18.05,18.49,18.00,257017,4.685382e+08,2.65,-2.49,-0.46,1.88
5,2025-05-29,600699,18.05,18.37,18.84,18.05,389412,7.193108e+08,4.38,1.77,0.32,2.85
6,2025-05-30,600699,18.27,17.77,18.29,17.65,324766,5.785653e+08,3.48,-3.27,-0.60,2.37
7,2025-06-03,600699,17.58,17.62,17.88,17.50,239206,4.218507e+08,2.14,-0.84,-0.15,1.75
8,2025-06-04,600699,17.54,17.70,17.78,17.53,169222,2.992779e+08,1.42,0.45,0.08,1.24
9,2025-06-05,600699,17.76,17.87,17.92,17.57,224576,3.998541e+08,1.98,0.96,0.17,1.64


In [13]:
# 2. 定义需要归一化的特征
features_to_normalize = ['开盘', '收盘', '最高', '最低', '振幅',]

# 3. 执行归一化
normalized_df, scalers, tensor_data = normalize_stock_data(stock_zh_a_hist_df, features_to_normalize)


In [14]:
normalized_df

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2025-05-22,600699,1.000000,1.000000,1.000000,1.000000,906208,1.782245e+09,1.000000,-3.53,-0.71,6.62
1,2025-05-23,600699,0.871166,0.943182,0.919771,0.913669,721013,1.407556e+09,0.992308,-0.77,-0.15,5.27
2,2025-05-26,600699,0.788344,0.840909,0.727794,0.827338,430418,8.185419e+08,0.428205,-1.40,-0.27,3.15
3,2025-05-27,600699,0.662577,0.670455,0.621777,0.597122,426355,7.866479e+08,0.802564,-2.37,-0.45,3.12
4,2025-05-28,600699,0.450920,0.496212,0.475645,0.514388,257017,4.685382e+08,0.441026,-2.49,-0.46,1.88
5,2025-05-29,600699,0.401840,0.617424,0.575931,0.532374,389412,7.193108e+08,0.884615,1.77,0.32,2.85
6,2025-05-30,600699,0.469325,0.390152,0.418338,0.388489,324766,5.785653e+08,0.653846,-3.27,-0.60,2.37
7,2025-06-03,600699,0.257669,0.333333,0.300860,0.334532,239206,4.218507e+08,0.310256,-0.84,-0.15,1.75
8,2025-06-04,600699,0.245399,0.363636,0.272206,0.345324,169222,2.992779e+08,0.125641,0.45,0.08,1.24
9,2025-06-05,600699,0.312883,0.428030,0.312321,0.359712,224576,3.998541e+08,0.269231,0.96,0.17,1.64


In [15]:
tensor_data.shape

torch.Size([45, 5])

In [1]:
import pandas as pd
import numpy as np
class StockDataProcessor:
    def __init__(self, lookback=10):
        self.lookback = lookback  # 用过去10天的数据预测未来1天
        self.scalers = {}  # 存储每只股票的归一化参数
    
    def normalize(self, df, features):
        """对数据进行Min-Max归一化，按股票代码分组处理"""
        normalized_df = df.copy()
        
        for stock_code, group in df.groupby('股票代码'):
            self.scalers[stock_code] = {}
            for feature in features:
                min_val = group[feature].min()
                max_val = group[feature].max()
                if max_val == min_val:
                    normalized = np.zeros_like(group[feature].values)
                else:
                    normalized = (group[feature] - min_val) / (max_val - min_val)
                
                self.scalers[stock_code][feature] = {'min': min_val, 'max': max_val}
                normalized_df.loc[group.index, feature] = normalized
        
        return normalized_df
    
    def create_sequences(self, df, features, target_close_col, target_pct_col):
        """创建        创建序列数据
        输入: 过去lookback天的特征
        输出: 未来1天的收盘价(回归目标)和涨跌幅分类(分类目标)
        """
        sequences = []
        targets_reg = []  # 回归目标：收盘价
        targets_cls = []  # 分类目标：涨跌幅类别
        
        # 按股票代码分组处理
        for stock_code, group in df.groupby('股票代码'):
            group = group.sort_values('日期')  # 确保数据按日期排序
            values = group[features].values
            close_values = group[target_close_col].values
            pct_values = group[target_pct_col].values
            
            # 创建序列
            for i in range(len(values) - self.lookback):
                seq = values[i:i+self.lookback]
                # 回归目标：未来1天的收盘价
                target_reg = close_values[i+self.lookback]
                # 分类目标：未来1天的涨跌幅类别
                target_pct = pct_values[i+self.lookback]
                
                # 将涨跌幅分为三类：跌、平、涨
                if target_pct < -0.01:  # 跌幅超0.01%
                    target_cls = 0
                elif target_pct > 0.01:  # 涨幅超0.01%
                    target_cls = 2
                else:  # 平
                    target_cls = 1
                
                sequences.append(seq)
                targets_reg.append(target_reg)
                targets_cls.append(target_cls)
        
        return np.array(sequences), np.array(targets_reg), np.array(targets_cls)

In [2]:
# 最小可运行项目：基于LSTM的股票走势预测系统
# 步骤：选股 + 技术指标生成 + 滑窗样本构建 + LSTM模型训练 + 预测 + 可视化 + 买入策略判断 + 数据保存

import pandas as pd
import numpy as np
import akshare as ak
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import os


In [21]:
###############################
# 1. 选股逻辑：获取沪深300中前10只股票（按代码排序）
###############################

stocks = ak.index_stock_cons(symbol="000300")
stocks = stocks.sort_values("品种代码").reset_index(drop=True)
selected_codes = list(stocks.loc[:19, '品种代码'])

In [22]:
selected_codes

['000001',
 '000001',
 '000002',
 '000063',
 '000063',
 '000100',
 '000157',
 '000157',
 '000166',
 '000301',
 '000333',
 '000338',
 '000408',
 '000425',
 '000425',
 '000538',
 '000568',
 '000568',
 '000596',
 '000617']

In [23]:
def fetch_and_process(code):
    df = ak.stock_zh_a_hist(symbol=code, period="daily", start_date="20150801", adjust="qfq")

    # df.to_csv(f"data/{code}_features.csv", index=False, encoding="utf-8-sig")
    return df

stock_dfs = [fetch_and_process(code) for code in selected_codes]
data = pd.concat(stock_dfs).reset_index(drop=True)

In [24]:
data

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2015-08-03,000001,7.36,7.88,7.90,7.32,1405123,1.757740e+09,7.73,5.07,0.38,1.19
1,2015-08-04,000001,7.84,7.91,8.02,7.69,774938,9.888435e+08,4.19,0.38,0.03,0.66
2,2015-08-05,000001,7.86,7.69,7.99,7.67,551802,7.045394e+08,4.05,-2.78,-0.22,0.47
3,2015-08-06,000001,7.61,7.64,7.80,7.55,450218,5.672085e+08,3.25,-0.65,-0.05,0.38
4,2015-08-07,000001,7.70,7.70,7.80,7.65,622010,7.855014e+08,1.96,0.79,0.06,0.53
...,...,...,...,...,...,...,...,...,...,...,...,...
47593,2025-08-05,000617,8.48,8.49,8.49,8.32,1828082,1.540071e+09,2.00,0.12,0.01,1.45
47594,2025-08-06,000617,8.48,8.46,8.55,8.37,1483568,1.254900e+09,2.12,-0.35,-0.03,1.17
47595,2025-08-07,000617,8.46,8.37,8.48,8.28,1439035,1.204531e+09,2.36,-1.06,-0.09,1.14
47596,2025-08-08,000617,8.38,8.34,8.46,8.32,1426095,1.196080e+09,1.67,-0.36,-0.03,1.13


In [28]:
data.to_csv('out19.csv', index=False, encoding="utf-8-sig")

In [20]:
for stock_code, group in data.groupby('股票代码'):
    print(f"股票代码: {stock_code}, 数据行数: {group.index}")

股票代码: 000001, 数据行数: Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')
股票代码: 000002, 数据行数: Index([14, 15, 16, 17, 18, 19, 20], dtype='int64')
股票代码: 000063, 数据行数: Index([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], dtype='int64')
股票代码: 000100, 数据行数: Index([35, 36, 37, 38, 39, 40, 41], dtype='int64')
股票代码: 000157, 数据行数: Index([42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55], dtype='int64')
股票代码: 000166, 数据行数: Index([56, 57, 58, 59, 60, 61, 62], dtype='int64')
股票代码: 000301, 数据行数: Index([63, 64, 65, 66, 67, 68, 69], dtype='int64')


In [ ]:
# 读取数据（替换为你的数据路径）
df = data
# 定义特征列
features = ['开盘', '收盘', '最高', '最低', '成交量', 
            '成交额', '振幅', '涨跌幅', "涨跌额"]

# 初始化数据处理器
lookback = 10  # 用过去10天数据预测未来1天
processor = StockDataProcessor(lookback=lookback)

# 归一化数据
normalized_df = processor.normalize(df, features)

/tmp/ipykernel_75736/4113369245.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.21586251 0.10565563 0.06663356 ... 0.09640709 0.1152492  0.13353415]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  normalized_df.loc[group.index, feature] = normalized


In [26]:
normalized_df

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2015-08-03,000001,0.120873,0.154782,0.142380,0.133672,0.215863,0.213024,0.474121,0.688754,0.593846,1.19
1,2015-08-04,000001,0.147734,0.156440,0.149055,0.154540,0.105656,0.109498,0.247923,0.520774,0.486154,0.66
2,2015-08-05,000001,0.148853,0.144279,0.147386,0.153412,0.066634,0.071219,0.238978,0.407593,0.409231,0.47
3,2015-08-06,000001,0.134863,0.141515,0.136819,0.146644,0.048869,0.052728,0.187859,0.483883,0.461538,0.38
4,2015-08-07,000001,0.139899,0.144831,0.136819,0.152284,0.078912,0.082120,0.105431,0.535458,0.495385,0.53
...,...,...,...,...,...,...,...,...,...,...,...,...
47593,2025-08-05,000617,0.370188,0.397887,0.367901,0.389904,0.189335,0.173542,0.100553,0.503546,0.492908,1.45
47594,2025-08-06,000617,0.370188,0.395246,0.372840,0.394256,0.153649,0.141400,0.106586,0.483938,0.478723,1.17
47595,2025-08-07,000617,0.368550,0.387324,0.367078,0.386423,0.149036,0.135723,0.118653,0.454318,0.457447,1.14
47596,2025-08-08,000617,0.361998,0.384683,0.365432,0.389904,0.147696,0.134771,0.083962,0.483521,0.478723,1.13


In [27]:
# 创建序列数据
sequences, targets_reg, targets_cls = processor.create_sequences(
    normalized_df, 
    features, 
    target_close_col='收盘', 
    target_pct_col='涨跌幅'
)

In [42]:
sequences.shape, targets_reg.shape, targets_cls.shape

((47448, 10, 9), (47448,), (47448,))

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [35]:
device

device(type='cuda')

In [38]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [36]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [37]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

In [39]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [40]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [41]:
from sklearn.model_selection import train_test_split

In [43]:
# 划分训练集、验证集和测试集
# 先划分训练集和临时集
X_train, X_temp, y_reg_train, y_reg_temp, y_cls_train, y_cls_temp = train_test_split(
    sequences, targets_reg, targets_cls, test_size=0.3, shuffle=False  # 时间序列不打乱
)

# 再将临时集划分为验证集和测试集
X_val, X_test, y_reg_val, y_reg_test, y_cls_val, y_cls_test = train_test_split(
    X_temp, y_reg_temp, y_cls_temp, test_size=1/3, shuffle=False  # 时间序列不打乱
)

In [46]:
X_train.shape, X_temp.shape, y_reg_train.shape, y_reg_temp.shape, y_cls_train.shape, y_cls_temp.shape

((33213, 10, 9), (14235, 10, 9), (33213,), (14235,), (33213,), (14235,))

In [50]:
from stock_predictor import StockPredictionModel
input_size = len(features)  # 输入特征数量
hidden_size = 64  # LSTM隐藏层大小
num_layers = 2  # LSTM层数
model = StockPredictionModel(input_size, hidden_size, num_layers)

In [51]:
model

StockPredictionModel(
  (lstm): LSTM(9, 64, num_layers=2, batch_first=True, dropout=0.2)
  (regression_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
  )
  (classification_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
  )
)

In [55]:
a = torch.FloatTensor(X_train[3:10])
a.shape

torch.Size([7, 10, 9])

In [56]:
out1, out2 = model(a)

In [58]:
out1.shape, out2.shape

(torch.Size([7]), torch.Size([7, 3]))